# 구글 리뷰 - 데이터 전처리 및 통합 감성 분석

1. 프로젝트 개요

'카카오톡 업데이트 사용자 피드백 분석' 프로젝트에서 두번째 단계로
수집된 원본 리뷰 데이터를 분석에 적합한 형태로 가공하고,
리뷰의 '감성'을 정교하게 분류하는 과정을 포함하였습니다.

2. 주요 프로세스 및 핵심 전략

신뢰성 높은 분석을 위해, 다음과 같은 체계적인 데이터 전처리 및 감성 분석 파이프라인을 구축했습니다.

- 데이터 통합 및 검증:
여러 시점에 걸쳐 수집된 리뷰 데이터를 하나의 데이터프레임으로 통합하고,
info(), isnull().sum(), to_datetime등을 통해 데이터의 정합성과 누락 여부를 꼼꼼하게 검증했습니다.

- 고급 텍스트 정제:
단순히 특수문자를 제거하는 것을 넘어, 분석에 중요한 '숫자'와 '영어' 정보는 보존하고, 의미 없는 이모티콘과 자음/모음 반복 등 '노이즈'만 선별적으로 제거하는 맞춤형 정제 함수(clean_text_with_lib)를 구현했습니다. 이는 분석 과정에서 핵심 정보가 소실되는 것을 방지하기 위한 전략적 선택이었습니다.

- '통합 감성 분석' 모델링:
단순히 별점(score)만으로 감성을 분류하는 방식의 한계를 인지하고,
'별점'과 'NLP(자연어 처리)'를 결합한 '통합 감성 분석' 모델을 적용했습니다.

  > (1단계) 1, 2, 4, 5점 리뷰는 명확한 긍/부정으로 우선 분류
  
  > (2단계) 판단이 모호한 3점 리뷰에 대해서만, 한국어 리뷰 데이터에 특화된 NLP 모델(matthewburke/korean_sentiment)을 적용하여 텍스트의 실제 맥락에 기반한 긍/부정 판단을 수행했습니다.

- 안정적인 대규모 처리:
수만 건에 달하는 3점 리뷰에 대한 NLP 분석은 상당한 시간이 소요되므로, '블록 단위 처리(Chunking)' 및 '주기적 중간 저장' 로직을 구현하여, 구글 코랩의 세션 중단 문제에 대응하고 데이터 유실 없이 안정적으로 작업을 완료할 수 있도록 파이프라인을 설계했습니다.

3. 결과물

모든 리뷰에 '통합 감성(sentiment_integrated)' 라벨이 부여된 최종 분석용 데이터셋(reviews_step3_integrated.pkl)이 생성되었고, '날짜별 감성 카운트/비율 추이' 그래프를 시각화하여, 이번 사태의 전반적인 여론 흐름에 대한 거시적인 인사이트를 확보했습니다.

In [ ]:
# 그래프 한글 나눔 폰트 설치 및 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 런타임 > 세션 다시 시작 실행

In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Matplotlib에서 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지

In [2]:
# 1. 데이터 병합 및 검증 작업
# 2차례에 거쳐 수집된 데이터를 병합하기 위해 호출
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/251005_kakao_reviews_final.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/251118_kakao_reviews_final.csv")

In [ ]:
# (필수) 수집 데이터의 날짜 타입을 동일하게 맞추기 위해
df1["at"] = pd.to_datetime(df1["at"])
print(df1.info())
df2["at"] = pd.to_datetime(df2["at"])
print(df2.info())

total_df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# score(별점) count 확인하기
print(total_df['score'].value_counts().sort_index())
print("===================================================")
# 날짜별 변화량 체크를 위해 date 컬럼 추가
total_df['date'] = total_df['at'].dt.date
print(total_df.info())

In [ ]:
# 날짜별 별점 count 체크
date_df = total_df['date']
grapeh_date = date_df.value_counts().sort_index()
print(grapeh_date)

In [ ]:
# 날짜별 카카오톡 구글 스토어 리뷰 등록 수(count)의 추이 그래프

plt.figure(figsize=(10, 6))
grapeh_date.plot(kind="bar", width =0.7)
plt.xlabel('날짜')
plt.ylabel('리뷰 수')
plt.title('날짜별 카카오톡 구글 스토어 리뷰 등록 수 추이')
plt.xticks(rotation=45)
tick_spacing = 5
plt.xticks(
    range(0, len(grapeh_date.index), tick_spacing),
    [date.strftime('%Y-%m-%d') for date in grapeh_date.index[::tick_spacing]],
    rotation=45,
    ha='right'
)
plt.tight_layout() # 그래프 요소들이 겹치지 않게 자동 조정
plt.show()

In [ ]:
# total_df를 파일로 저장하여 중간 결과 백업
file_name = 'total_reviews_251119.pkl'

# 코랩 임시 저장소에 저장
save_path = file_name

total_df.to_pickle(save_path)
print(f"'{save_path}' 경로에 통합된 데이터가 성공적으로 저장되었습니다.")

In [ ]:
# 2. 텍스트 정제 작업
# 이모지 제거를 위한 인스톨
!pip install emoji

In [ ]:
# 자연어 분석을 위한 함수 만들기 & 테스트 (테스트 꼭 하고 넘기기)
import re
import emoji # 방금 설치한 라이브러리를 불러옵니다.

def clean_text_with_lib(text):
    # 입력값을 문자열로 변환
    text = str(text)

    # 1. [수정됨] emoji 라이브러리를 사용하여 모든 이모티콘을 제거합니다.
    # replace_emoji 함수는 텍스트 안의 모든 이모티콘을 찾아 '' (빈 문자열)로 바꿔줍니다.
    text = emoji.replace_emoji(text, replace='')

    # 2. 한글, 영어, 숫자를 제외한 특수문자 제거
    text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)

    # 3. 자음 또는 모음만 반복되는 경우 제거 (ㅋㅋㅋ, ㅜㅜㅜ)
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1{2,}', '', text)

    # 4. 여러 개의 공백을 하나의 공백으로 변경
    text = re.sub(r'\s+', ' ', text)

    # 5. 앞뒤 공백 제거
    text = text.strip()

    return text

# --- 함수 적용 테스트 ---
test_review1 = "업데이트 최악 1점 드립니다.ㅠㅠㅠ UI 돌려놔!!!! #카카오"
test_review2 = "ㅋㅋㅋ Only salvation is Jesus, believe in Jesus 🙏"
test_review3 = "이모티콘😄은 좋은데, 앱 속도가 너무 느려요..."
test_review4 = "0점이 없어서 1점남김ㅋㅋ"

print(f"원본: {test_review1} -> 정제 후: {clean_text_with_lib(test_review1)}")
print(f"원본: {test_review2} -> 정제 후: {clean_text_with_lib(test_review2)}")
print(f"원본: {test_review3} -> 정제 후: {clean_text_with_lib(test_review3)}")
print(f"원본: {test_review4} -> 정제 후: {clean_text_with_lib(test_review4)}")

In [ ]:
# 실 데이터 반영
from tqdm.auto import tqdm

# 1. apply()를 사용하여 content 컬럼의 각 리뷰에 함수를 적용
print("텍스트 정제 작업을 시작합니다... (데이터 양에 따라 시간이 소요됩니다)")
total_df['cleaned_content'] = total_df['content'].apply(clean_text_with_lib)
print("텍스트 정제 작업 완료!")

print("\n--- [ 텍스트 정제 후 결과 확인 ] ---")
print(total_df[['content', 'cleaned_content']].head())

# 3. 정제된 내용 중 빈 경우가 있는지 확인
empty_count = total_df['cleaned_content'].str.len().eq(0).sum()
print(f"\n정제 후 내용이 비어있는 리뷰의 수: {empty_count} 개")

In [ ]:
# 3. 통합 감성 분석' 모델링 작업
# 필요 라이브러리 설치 및 임포트
!pip install transformers torch sentencepiece

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
# 3-1. 별점 기반 '기본 감성' 분류
def classify_sentiment_simple(score):
    if score <= 2: return 'Negative'
    elif score == 3: return 'Neutral'
    else: return 'Positive'

print("1단계: 별점 기반 기본 감성 분류를 시작합니다...")
total_df['sentiment_simple'] = total_df['score'].apply(classify_sentiment_simple)

# 중간 결과 저장 1
total_df.to_pickle("reviews_step1_simple_sentiment.pkl")
print("✅ 1단계 완료 및 'reviews_step1_simple_sentiment.pkl' 파일로 저장 완료!")

In [ ]:
# 3-2. 3점 리뷰 NLP 분석 (블록 단위 처리 및 자동 저장)
print("\n2단계: 3점 리뷰 NLP 분석을 시작합니다... (중간중간 자동 저장됩니다)")

# NLP 모델 로드
MODEL_NAME = "matthewburke/korean_sentiment"
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    NLP_READY = True
except Exception as e:
    print(f"모델 로딩 실패: {e}"); NLP_READY = False

# NLP 분석 함수
def get_sentiment_nlp_final(text):
    # (이전 답변의 함수와 동일)
    if not isinstance(text, str) or not text.strip(): return 'Neutral'
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad(): outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment = model.config.id2label[torch.argmax(predictions).item()]
        return sentiment
    except: return 'Neutral'

# 블록 단위 처리 및 저장
if NLP_READY:
    # 3점 리뷰의 인덱스만 가져오기
    neutral_indices = total_df[total_df['score'] == 3].index
    chunk_size = 500  # ★ 500개씩 처리하고 저장 (컴퓨터 사양에 따라 조절)

    # NLP 결과를 저장할 새로운 컬럼 초기화
    total_df['sentiment_nlp'] = 'Not_Applicable'

    # tqdm으로 전체 진행 상황 표시
    for i in tqdm(range(0, len(neutral_indices), chunk_size), desc="3점 리뷰 NLP 처리 중"):
        chunk_indices = neutral_indices[i:i + chunk_size]

        # 현재 블록에 대해 NLP 분석 실행
        nlp_results = total_df.loc[chunk_indices, 'cleaned_content'].apply(get_sentiment_nlp_final)

        # 결과를 원본 데이터프레임에 업데이트
        total_df.loc[chunk_indices, 'sentiment_nlp'] = nlp_results

        # --- 중간 결과 저장 2 ---
        total_df.to_pickle("reviews_step2_nlp_inprogress.pkl")

    print("\n✅ 2단계 완료 및 'reviews_step2_nlp_inprogress.pkl' 파일로 최종 저장 완료!")
else:
    print("NLP 모델이 준비되지 않아 2단계를 건너뜁니다.")

In [ ]:
# 3-3. '통합 감성' 생성
try:
    df_with_nlp = pd.read_pickle("reviews_step2_nlp_inprogress.pkl")

    def integrate_sentiment_final(row):
        if row['score'] <= 2: return 'Negative'
        elif row['score'] >= 4: return 'Positive'
        else: # score == 3
            return row['sentiment_nlp'] if row['sentiment_nlp'] in ['Positive', 'Negative'] else 'Neutral'

    print("\n3단계: 최종 감성 통합을 시작합니다...")
    df_with_nlp['sentiment_integrated'] = df_with_nlp.apply(integrate_sentiment_final, axis=1)

    # 중간 결과 저장 3
    df_with_nlp.to_pickle("reviews_step3_integrated.pkl")
    print("✅ 3단계 완료 및 'reviews_step3_integrated.pkl' 파일로 저장 완료!")

except FileNotFoundError:
    print("오류: 2단계 결과 파일이 없습니다. 2단계를 먼저 실행해주세요.")

In [ ]:
# 3-4. 최종 결과 확인 및 시각화
try:
    final_df = pd.read_pickle("reviews_step3_integrated.pkl")

    print("\n--- [ 최종 통합 감성 분석 결과 ] ---")
    print(final_df['sentiment_integrated'].value_counts())

    # 날짜별 100% 비율 누적 막대그래프 시각화
    final_sentiment_counts = final_df.groupby(['date', 'sentiment_integrated']).size().unstack(fill_value=0)
    final_sentiment_proportions = final_sentiment_counts.div(final_sentiment_counts.sum(axis=1), axis=0)

    final_sentiment_proportions.plot(kind='bar', stacked=True, figsize=(10, 6), width=0.7)
    plt.title('날짜별 최종 통합 감성 비율(%) 추이', fontsize=14, pad=20)
    plt.xlabel('날짜'); plt.ylabel('감성 비율')
    plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0], ['0%', '20%', '40%', '60%', '80%', '100%'])
    tick_spacing = 5
    plt.xticks(range(0, len(final_sentiment_proportions.index), tick_spacing),
               [date.strftime('%Y-%m-%d') for date in final_sentiment_proportions.index[::tick_spacing]],
               rotation=45, ha='right')
    plt.legend(title='통합 감성', bbox_to_anchor=(0.87, 1.3), loc='upper left')
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print("오류: 3단계 결과 파일이 없습니다. 3단계를 먼저 실행해주세요.")

In [ ]:
# '카운트' 기반 누적 막대그래프 시각화

print("\n--- [ 날짜별 최종 통합 감성 '카운트' 추이 ] ---")

final_sentiment_counts.plot(kind='bar', stacked=True, figsize=(10, 6), width=0.7) # 색상은 동일하게 유지

plt.title('날짜별 최종 통합 감성 카운트(수) 추이', fontsize=14, pad=20)
plt.xlabel('날짜')
plt.ylabel('리뷰 수')

tick_spacing = 5
plt.xticks(range(0, len(final_sentiment_counts.index), tick_spacing),
           [date.strftime('%Y-%m-%d') for date in final_sentiment_counts.index[::tick_spacing]],
           rotation=45, ha='right')
plt.legend(title='통합 감성', loc='upper right')
plt.tight_layout()
plt.show()